In [1]:
import numpy as np
# Load the "autoreload" extension so that code can change
%load_ext autoreload

#always reload modules so that as you change code in src, it gets loaded
%autoreload 2

from src.data import make_dataset

In [4]:
path = "/Users/jbaripatti/machine_learning_proj/data/interim/"
name = 'data_interim.txt'
filepath = path + name
data = np.loadtxt(filepath, delimiter=",")
X_train, y_train, X_test, y_test = make_dataset.split_data(data)

In [5]:
import tensorflow as tf
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Dropout
from keras import regularizers
from keras.models import Sequential

ModuleNotFoundError: No module named 'tensorflow'